<a href="https://colab.research.google.com/github/usha333/AI-Speech-WebApp/blob/main/AI_speech_WebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AI Speech App**

Importing Libraries



In [1]:
import numpy as np
import random
import json
import nltk
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Creating Custom Functions:

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

Stemming

In [3]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

Bag of Words:

In [4]:
def bag_of_words(tokenized_sentence, words):
   
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

Loading the Data & Cleaning it

In [5]:
with open('/content/drive/MyDrive/ai web app/intents.json', 'r') as f:
   intents = json.load(f)

In [6]:
all_words = []
tags = []
xy = []
# looping through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # adding to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenizing each word in the sentence
        w = tokenize(pattern)
        # adding to our words list
        all_words.extend(w)
        # adding to xy pair
        xy.append((w, tag))

Implementing custom Functions

In [7]:
# stemming and lowering each word

ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]

# removing duplicates and sorting

all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

12 patterns
5 tags: ['goodbye', 'greeting', 'hru', 'name', 'thanks']
20 unique stemmed words: ["'s", 'a', 'are', 'bye', 'goodby', 'hello', 'help', 'hey', 'hi', 'how', 'is', 'later', 'lot', 'name', 'see', 'thank', 'that', 'what', 'you', 'your']


Creating Training Data:

In [8]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1.]]


PyTorch Model Creating Model

In [9]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

Assigning the Dataset to the Model:

In [10]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

Hyper Parameters:

In [11]:
# Hyper-parameters is a parameter whose value is used to control the learning process
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

20 5


Loss and optimizer:

In [12]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                         batch_size=batch_size,
                         shuffle=True,
                         num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Training the Model

In [13]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

Epoch [100/1000], Loss: 1.0885
Epoch [200/1000], Loss: 0.8047
Epoch [300/1000], Loss: 0.4427
Epoch [400/1000], Loss: 0.0404
Epoch [500/1000], Loss: 0.0187
Epoch [600/1000], Loss: 0.0160
Epoch [700/1000], Loss: 0.0061
Epoch [800/1000], Loss: 0.0119
Epoch [900/1000], Loss: 0.0050
Epoch [1000/1000], Loss: 0.0030
final loss: 0.0030


Saving the Trained Model

In [14]:
FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


Loading our Saved Model

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('/content/drive/MyDrive/ai web app/intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=20, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=5, bias=True)
  (relu): ReLU()
)

In [16]:
#pip install wikipedia
#import wikipedia
#query=input("enter your query:")
#page=wikipedia.page(query)
#print(page.summary)

In [17]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11697 sha256=b5e496825b7d0e0f9574074e5b1c4275b51ab105824186be7d3769d4d8021173
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [18]:
import wikipedia

In [22]:
!mkdir my_project

In [20]:
#/content/my_project
#/content/myprojectchatbot1.text

 **Using the Chatbot:**

In [ ]:
bot_name = "bot"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    #open('chatbot3.text','a')
    outfile=open('/content/my_project/myprojectchatbot2.text','a')
    sentence1= input("You: ")
    #outfile.write(sentence1+ ':')
    if sentence1 == "quit":
        break

    sentence = tokenize(sentence1)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.99:
      for intent in intents['intents']:
        if tag == intent["tag"]:
          outfile.write(sentence1+ ':')
          print(f"{bot_name}:: {random.choice(intent['responses'])}")
          print(f"{bot_name}::{random.choice(intent['responses'])}",file=outfile)
    elif prob.item() < 0.50:
      with open('/content/my_project/myprojectchatbot2.text',"r") as outfile:
        lines=outfile.readlines()
        alpha='::'
        for i in lines:
          if sentence1 in i:
            outfile.write(sentence1+ ':')
            ans=i.partition(alpha)[2]
            print(f"{bot_name}:"+ ans)
            #print(sentence)
        
    else:
      outfile.write(sentence1+ ':')
      print(f"{bot_name}::")
      print(wikipedia.summary(sentence,sentences=1))
      print(":"+wikipedia.summary(sentence,sentences=1),file=outfile)
            
      

outfile.close()

Let's chat! (type 'quit' to exit)
You: Hi
bot:: Hello
You: What is your name
bot::
A name is a term used for identification by an external observer.
